In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.7.7/install/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import torch
from marian_mt import MarianMTModel
from transformers import MarianTokenizer, MarianConfig


MESSAGE FROM RCS: Setting LC_ALL and LANG to locale value 'en_US.utf8'.  If you wish to use a different locale, use the following code to set it to your prefered locale prior to importing click.
###############################
import os
os.environ["LC_ALL"]="new locale"
os.environ["LANG"]=os.environ["LC_ALL"]
###############################



In [4]:
romance_model_name   = 'Helsinki-NLP/opus-mt-afa-en'
rmt = MarianMTModel.from_pretrained(romance_model_name, output_attentions=True)

In [5]:
romance_tokenizer = MarianTokenizer.from_pretrained(romance_model_name)

In [6]:
#based on the coverage penalty equation found here https://arxiv.org/pdf/1807.11243.pdf
def coverage_penalty(n_heads, n_layers, bs, slen, dec_attention, tlength):
    # dec_attention: n_layers x (bs,n_heads,slen,slen)

    tlen = tlength[:, :, :, 0]  # view of the tlength tensor of sentence lengths, of shape: (n_layers, bs, n_heads)
    t_ones = torch.ones([n_layers, bs, n_heads, slen], dtype=torch.float32)
    t_sum = torch.zeros([n_layers, bs, n_heads, slen], dtype=torch.float32)
    for i in range(n_layers):
        t_sum[i] = torch.sum(dec_attention[i], dim=2)  #sum of the target weights over one src sentence token from all tgt sentence tokens
    t_min = torch.min(t_sum, t_ones)  #min (sum,1)
    t_min = torch.add(t_min, 1e-8) #adding small epsilon to avoid log nans
    t_log = torch.log(t_min)    #calculate the logarithm
    for i in range(bs):   #setting to 0 any positions > original sentence length   (log0 would penalize these o/w)
        lsent = int(tlen[0,i,0])
        t_log[:,i,:,lsent:]=0
    t_sum_log = torch.sum(t_log, dim=3) #sum over all source tokens of src sentence
    covp = - torch.div(t_sum_log, tlen)  # divide by |x|= sentence length
    return covp

In [7]:
import pandas as pd
# import glob

# csv = glob.glob('drive/My Drive/Colab Notebooks/extracted.csv')
# print(csv)
df = pd.read_csv('new_train.csv')

In [8]:
# df = df[pd.notnull(df['source'])]
# df = df.drop([df.index[4200],df.index[4201]])
# print(df.head)
print(df.columns)
# df = df.truncate(after=99999)
source_sent = df['source'].values
target_sent = df['target'].values
print(source_sent[0])

Index(['Unnamed: 0', 'source', 'target'], dtype='object')
Daanyeer ama Muusekorow , waa bahal ku nool caalamka , waxoo u badanyahay qaaradaha afrika iyo aasiya.Daanyeerka waa bahal geedaha fuulo , waxoona u dhisanyahay sida biniaadanka.



In [9]:
print(len(source_sent))

14328


In [10]:

# print(source_sent[4201]) dropping 4200 4201
print(source_sent[4201])

waxa soomaali kuugu soo turjumay MOHAMED HUSSEIN oo ku nool magaalada hargeysa kuna magac dheer Mohamed aasif oo dhigta jaamacada caalamigaah ee Abaarso Tech University Hargeisa Compus



In [11]:
print(target_sent[4201])

Linux ( or ) is a family of open source Unix-like operating systems based on the Linux kernel , an operating system kernel first released on September 17 , 1991 , by Linus Torvalds.



In [12]:
# source_sent = source_sent[:-1]
# target_sent = target_sent[:-1]
print(len(source_sent),len(target_sent))
# source_sent_u = set(source_sent)
# target_sent_u = set(target_sent)
# print(len(source_sent_u),len(target_sent_u))

14328 14328


In [13]:
from torch.utils.data import Dataset, DataLoader
marian_loader = DataLoader(source_sent,batch_size=8, shuffle=False)

In [14]:
for data in marian_loader:
    for text in data:
        print(text)
    print(len(data))
    break
    

Daanyeer ama Muusekorow , waa bahal ku nool caalamka , waxoo u badanyahay qaaradaha afrika iyo aasiya.Daanyeerka waa bahal geedaha fuulo , waxoona u dhisanyahay sida biniaadanka.

Apple INC waa shirkad mareykan ah oo samaysa kumbiyuutarada iyo talefoonada gacanta iyo qalabada MP3 Player ah oo muskigga laga dhageysto oo loo yaqaan Ipod.

Jaamacada Carabta waa urur kulmiya wadamada carabta.Waa urur kulmiya wadamo kuyaala Afrika iyo Aasiya xubnaha kujira waxaa looyaqaana dawldo caraba.waa dawlado wadaaga arimo dhaqaale iyo arimo siyaasadeed.

waxaana ka dhexeeya xidhiidho aad iyo aad ubadan

waxee ku taalaa meesha ee afrika iyo aasiya ee iskaga dhagaayaan oo ah Siinaa.dalka ugu weyn jasiirad dhuleedkaan waa Sacuudi Carabiya , waxeena daris la tahay dallalka kala ah : -

Bada Carbeed waa magac loo bixiyay biyaha Bariga Dhexe oo ka mid ah Badweynta Hindiya.

Waxay ka mid tahay magaalooyinka ugu beeraha badan jamhuuriyada Somaliland.

Arktik ( " loogu dhawaaqo " : ) waa deegaanada cidhifka u

In [15]:
bsz = 8
scores = []
n_heads = 8
n_layers = 6
counter=0
for data in marian_loader:
#     print(data)
#     src_texts = [text for text in data]
#     print(src_texts)
    batch = romance_tokenizer.prepare_seq2seq_batch(data)
    # print(batch)
    outputs = rmt(**batch, return_dict=False)
    dec_attn=[]
    for i in range(n_layers):
        dec_attn += (outputs[i][:,:,:,:].detach().clone(),)
    
    #print(dec_attn[1].shape) 
    len2=batch['input_ids'].shape[1] 
    #print(len2)
#     sentences_list = romance_tokenizer(src_texts,padding=False)['input_ids']
    sentences_list = romance_tokenizer(data,padding=False)['input_ids']
    len1 = [len(item) -1 for item in sentences_list]  
    #print(len1)
    tlength = torch.zeros(6, bsz, 8, len2)
    for i in range(bsz):
        tlength[:, i, :, :] = len1[i]
    #print(tlength)
    cp = coverage_penalty(n_heads, n_layers, bsz, len2, dec_attn, tlength)
#     print(cp.shape)
    cp = torch.mean(cp.float(), dim=0)
    cpd = cp.numpy()
#     print(cpd.shape)
#     print(cpd)
    for i in range(bsz):
        scores.append(cpd[i,:])
    counter = counter +1
    if(counter%5000==0):
        print(counter)
#     break
#     if(counter>520):
#         print(counter)
    #print(cp.shape)  


Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors


In [16]:
import numpy as np
scores = np.asarray(scores)
print(len(scores))
print(scores[0])

14328
[2.3674135 2.896929  2.3316152 2.2963922 2.1594524 2.2254715 2.1232684
 2.5464475]


In [17]:
np.savetxt("somali_scores.csv", scores, delimiter=",")

In [18]:
# d = {'source':source_sent[:97400],'target':target_sent[:97400]}
# df2 = pd.DataFrame(data=d)
# df2.to_csv('smaple.')